# Extracción de perfiles a partir de about

In [14]:
#LIBRARIES
import pandas as pd
import re
import numpy as np

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## nomalice parameters
from unidecode import unidecode

In [15]:
df = pd.read_csv("DF_BOOTCAMPS_2591.csv")

In [16]:
dfa = df[df["z"].notna()] 
dfa = dfa.reset_index(drop = True) 

In [17]:
dfa.shape

(1423, 29)

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE ABOUT

In [18]:
#CONVERTIR EN STRING
dfa["about"] =  dfa["about"].str.lower()

In [19]:
#NOMBRE DE BOOTCAMPS
lista_boots = ['4Geeks',
 'Acámica',
 'Ada ITW',
 'Bedu Tech',
 'Bogotá Dev',
 'Bootcamp Institute',
 'Coderise International',
 'Desafío Latam',
 'Dev F.',
 'Digital House',
 'Generation Mexico',
 'Hack Academy',
 'HolaCode',
 'Holberton School',
 'iCreate Coding Academy',
 'IJALTI',
 'Iron Hack',
 'Jóvenes a Programar',
 'Keep Coding',
 'Kod1go',
 'Laboratoria',
 'Le Wagon',
 'Make it Real Camp',
 'MindHub ',
 'Muktek Academy',
 'Nivel Pro',
 'Noukod',
 'Plataforma 5',
 'Programa Valentina',
 'Pygmalion',
 'Reprograma',
 'World Tech Inc (a.k.a. World Tech Makers –WTM–)',
 'BICTIA',
 'Talento Digital']

In [20]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'pygmalion',
 'reprograma',
 'worldtech$',  #se cambió  para que solo termine con $
 'bictia|bi.?tia']

In [21]:
#BUSQUEDA CON REGEX
c = []
for boot in range(len(boots)): 
    for i in range(len(dfa.about)): 
        A = re.findall(boots[boot], dfa.about[i])
        if len(A) > 0:
            c.append((dfa.id[i], boots[boot], lista_boots[boot]))

            # print(f"{dfa['job_title'][i]}---{A[0]}---{boots[boot]}--- \n {dfa.about[i]} \n ------------------------------------------\n")


#CREAR DATAFRAME
c = list(set(c))
about_regex = pd.DataFrame.from_records(c, columns=['id', 'regex_bootcamp', 'bootcamp'])

#HACER MERGE CON EL RESTO DE COLUMNAS Y EXPORTAR 
# about_regex = pd.merge(about_regex, dfa, "left", "id")

about_regex.to_excel("2_about_regex.xlsx", index = False)

In [22]:
#BUSQUEDA CON FUZZY PARTIAL
d = []
for boot in range(len(lista_boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfa.about)): #PARA CADA INDIVIDUO
        bootCAMP = lista_boots[boot]
        bootCAMP = bootCAMP.lower()
        bootCAMP = unidecode(bootCAMP)
        norEdu =  unidecode(dfa.about[i])
        ## se hace un aproach a fuzzy
        wordLen = len(dfa.about[i])
        ratio = fuzz.partial_ratio(bootCAMP,norEdu)
        #ratio = fuzz.token_sort_ratio(bootCAMP,norEdu)
        #ratio = fuzz.token_set_ratio(bootCAMP,norEdu)
        #print(ratio)
        if ratio>85 and wordLen >=5:
            d.append((dfa.id[i], boots[boot], lista_boots[boot], ratio))

#CREAR DATAFRAME
d = list(set(d))
about_fp = pd.DataFrame.from_records(d, columns=['id', 'regex_bootcamp', 'bootcamp', 'ratio'])

#HACER MERGE CON EL RESTO DE COLUMNAS Y EXPORTAR 
# about_fp = pd.merge(about_fp, dfa, "left", "id")

about_fp.to_excel("2_about_fp.xlsx", index = False)

In [28]:
#BUSQUEDA CON FUZZY SET
k = []
for boot in range(len(lista_boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfa.about)): #PARA CADA INDIVIDUO
        bootCAMP = lista_boots[boot]
        bootCAMP = bootCAMP.lower()
        bootCAMP = unidecode(bootCAMP)
        norEdu =  unidecode(dfa.about[i])
        ## se hace un aproach a fuzzy
        wordLen = len(dfa.about[i])
        # ratio = fuzz.partial_ratio(bootCAMP,norEdu)
        # ratio = fuzz.token_sort_ratio(bootCAMP,norEdu)
        ratio = fuzz.token_set_ratio(bootCAMP,norEdu)
        #print(ratio)
        if ratio>85 and wordLen >=5:
            try:
                k.append((dfa.id[i], boots[boot], lista_boots[boot], ratio))
            except:
                None

#CREAR DATAFRAME
k = list(set(k))
about_fs = pd.DataFrame.from_records(k, columns=['id', 'regex_bootcamp', 'bootcamp', 'ratio'])

#HACER MERGE CON EL RESTO DE COLUMNAS Y EXPORTAR 
# about_fp = pd.merge(about_fp, dfa, "left", "id")

about_fs.to_excel("2_about_fs.xlsx", index = False)

In [24]:
k

[(32690217, 'digital[ -]?house', 'Digital House', 100),
 (46810892, 'laborat.ria ', 'Laboratoria', 100),
 (47818340, 'laborat.ria ', 'Laboratoria', 100),
 (48426552, 'laborat.ria ', 'Laboratoria', 100),
 (49553788, 'laborat.ria ', 'Laboratoria', 100),
 (50136139, 'laborat.ria ', 'Laboratoria', 100),
 (50558792, 'laborat.ria ', 'Laboratoria', 100),
 (51112086, 'laborat.ria ', 'Laboratoria', 100),
 (53238040, 'laborat.ria ', 'Laboratoria', 100),
 (67980737, 'laborat.ria ', 'Laboratoria', 100),
 (74733633, 'laborat.ria ', 'Laboratoria', 100),
 (79419003, 'laborat.ria ', 'Laboratoria', 100),
 (14161321, 'nivel[ -]?pro ', 'Nivel Pro', 100),
 (50185459, 'nivel[ -]?pro ', 'Nivel Pro', 100),
 (50606757, 'nivel[ -]?pro ', 'Nivel Pro', 100),
 (13566127, 'plataforma[ -]?5', 'Plataforma 5', 91),
 (15614664, 'plataforma[ -]?5', 'Plataforma 5', 91),
 (17538393, 'plataforma[ -]?5', 'Plataforma 5', 91),
 (28845407, 'plataforma[ -]?5', 'Plataforma 5', 91),
 (44032888, 'plataforma[ -]?5', 'Plataforma 5'